In [1]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
%pip install transformers

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
len(tokenizer.vocab)


30522

In [4]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [8]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [9]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [10]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [12]:
%pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html

Note: you may need to restart the kernel to use updated packages.Looking in links: https://download.pytorch.org/whl/torch_stable.html

  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.0%2Bcu118-cp39-cp39-win_amd64.whl (2611.4 MB)

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\anaconda\\Lib\\site-packages\\~.rch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


In [13]:
%pip install torch trochvision

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement trochvision (from versions: none)
ERROR: No matching distribution found for trochvision


In [14]:
%pip install torchtext==0.9.0

  Using cached torch-1.8.0-cp39-cp39-win_amd64.whl (190.5 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.8.0 which is incompatible.



  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118


In [12]:
from torchtext.legacy import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [13]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [14]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [15]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['the', 'premise', 'of', 'cabin', 'fever', 'starts', 'like', 'it', 'might', 'have', 'something', 'to', 'offer', '.', 'a', 'group', 'of', 'college', 'teens', 'after', 'finals', '(', 'in', 'the', 'fall', '?', ')', 'goes', 'to', 'a', 'resort', 'cabin', 'in', 'the', 'woods', 'where', 'one', 'by', 'one', 'they', 'are', 'attacked', 'by', 'an', 'unseen', 'flesh', 'eating', 'virus', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'unfortunately', ',', 'the', 'first', 'paragraph', 'is', 'where', 'any', 'remote', 'elements', 'of', 'film', 'quality', 'stop', '.', 'cabin', 'fever', 'is', 'little', 'more', 'than', 'college', 'kids', 'looking', 'for', 'sex', ',', 'boo', '##ze', ',', 'talking', 'non', '-', 'stop', 'about', 'nothing', ',', 'and', 'seeing', 'how', 'many', 'f', '-', 'bombs', 'they', 'can', 'get', 'into', '1', ':', '40', 'minutes', 'or', 'however', 'long', 'this', 'mess', 'is', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'kids', 'act', 'and', 'react', 'stupid', '##ly', 'to', 'ev

In [16]:
LABEL.build_vocab(train_data)

In [17]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [18]:
### build model
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [20]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [22]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [24]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [25]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [26]:
criterion = nn.BCEWithLogitsLoss()

In [27]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [28]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            print("epoch_acc: ", epoch_acc)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
# def evaluate(model, iterator, criterion, device):
#     epoch_loss = 0
#     epoch_acc = 0
#     model.eval()

#     with torch.no_grad():
#         for batch in iterator:
#             # Move the batch to the GPU
#             inputs = batch.text.to(device)
#             targets = batch.label.to(device)

#             # Compute the model's output
#             predictions = model(inputs).squeeze(1)

#             # Compute the loss and accuracy
#             loss = criterion(predictions, targets)
#             acc = binary_accuracy(predictions, targets)

#             # Update the loss and accuracy
#             epoch_loss += loss.item()
#             epoch_acc += acc.item()

#     # Compute the average loss and accuracy
#     num_batches = len(iterator)
#     epoch_loss /= num_batches
#     epoch_acc /= num_batches

#     return epoch_loss, epoch_acc

In [30]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [30]:
# model_save_name = 'classifier.pt'
# path = F"C:/Users/ammar/Downloads/{model_save_name}"

In [ ]:
# import os


# if(os.path.exists(path)):
#     if(torch.cuda.is_available()):
#         model.load_state_dict(torch.load(path, map_location=torch.device('cuda')))
#         test_loss, test_acc = evaluate(model, test_iterator, criterion)
#         print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
#     else:
#         print("No GPU found")
#         model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
#         test_loss, test_acc = evaluate(model, test_iterator, criterion)
#         print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
#         #save model for future use
#         torch.save(model.state_dict(), path)

In [18]:
# %pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [31]:
import flask
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    model.eval()
    sentence = request.json['sentence']
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(tensor)
    prediction = torch.sigmoid(output).item()
    return jsonify({'sentiment_score': prediction})

if __name__ == '__main__':
    app.run(port=4000)
    #print something to know it's running
    if(app.run(port=4000)):
        print("running")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)


In [ ]:
predict_sentiment(model, tokenizer, "i don't like this hostel")

0.08202903717756271

In [ ]:
predict_sentiment(model, tokenizer, "hostel was good")

0.909687340259552

In [ ]:
df = pd.read_csv("drive/MyDrive/fast_thesis/clean_hotel_reviews.csv")

In [ ]:
t_new_rate=[]
for ind in df.index:
  score = predict_sentiment(model, tokenizer, df['reviews.text'][ind])
  x_std = (score - 0) / (1 - 0)
  x_scaled = x_std * (5 - 0) + 0
  t_new_rate.append(round(x_scaled))